In [86]:
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize

In [87]:
def create_raw_token_map(tokens):
    raw_token_map_array = np.zeros(shape=(len(tokens), len(tokens)))
    raw_token_map = pd.DataFrame(raw_token_map_array, columns=tokens, index=tokens)
    return raw_token_map

In [3]:
def add_text(raw_token_map, text, token_set):
    token_set = raw_token_map.index
    text = text.lower()
    tokens = word_tokenize(text)
    pointer = 0
    while pointer < len(tokens) - 1:
        current_token = tokens[pointer]
        if current_token in token_set:
            weight = 10
            subpointer = pointer + 1
            while weight > 0 and subpointer < len(tokens):
                next_token = tokens[subpointer]
                if next_token in token_set:
                    raw_token_map.at[current_token, next_token] += weight
                    raw_token_map.at[next_token, current_token] += weight
                weight -= 1
                subpointer += 1
        pointer += 1
        if pointer % 500 == 0:
            print(pointer)

In [4]:
def standardize_token_map(raw_token_map):
    token_map = raw_token_map.div(raw_token_map.sum(axis=1), axis=0)
    token_map = token_map.fillna(0)
    doubled_words = set()
    for word1 in raw_token_map.index:
        for word2 in raw_token_map.columns:
            if word1 != word2 or word1 not in doubled_words:
                word1_score = token_map.at[word1, word2]
                word2_score = token_map.at[word2, word1]
                score = word1_score * word2_score
                token_map.at[word1, word2] = score
                if word1 == word2:
                    doubled_words.add(word1)
    return token_map

In [5]:
with open('1000-most-common-words.txt', 'r') as f:
    token_list = f.readlines()
token_set = set([token.lower().strip() for token in token_list])
token_list = list(token_set)

In [6]:
with open('count_of_monte_cristo.txt', 'r') as f:
    text = f.read()

In [7]:
raw_token_map = create_raw_token_map(token_set)

In [8]:
add_text(raw_token_map, text, token_set)

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
30000
30500
31000
31500
32000
32500
33000
33500
34000
34500
35000
35500
36000
36500
37000
37500
38000
38500
39000
39500
40000
40500
41000
41500
42000
42500
43000
43500
44000
44500
45000
45500
46000
46500
47000
47500
48000
48500
49000
49500
50000
50500
51000
51500
52000
52500
53000
53500
54000
54500
55000
55500
56000
56500
57000
57500
58000
58500
59000
59500
60000
60500
61000
61500
62000
62500
63000
63500
64000
64500
65000
65500
66000
66500
67000
67500
68000
68500
69000
69500
70000
70500
71000
71500
72000
72500
73000
73500
74000
74500
75000
75500
76000
76500
77000
77500
78000
78500
79000
79500
80000
80500
81000
81500
82000
82500
83000
83500
84000
84500
85000


In [9]:
token_map = standardize_token_map(raw_token_map)

In [83]:
def sample_from_related_tokens(token_map, size, token):
    token_row = token_map.loc[token, :].squeeze(axis=0)
    probabilities = token_row / token_row.sum()
    sampled_indices = np.random.choice(len(probabilities), size=size, replace=False, p=probabilities)
    sampled_probabilities = [probabilities[i] for i in sampled_indices]
    sampled_tokens = [token_list[i] for i in sampled_indices]
    return sampled_tokens

In [84]:
print(sample_from_related_tokens(token_map, 5, 'and'))
print(token_map.loc['and', :].squeeze(axis=0).sort_values(ascending=False)[:5])

['sudden', 'gray', 'shoe', 'level', 'wall']
the    0.004463
of     0.001685
his    0.001516
to     0.001455
in     0.000920
Name: and, dtype: float64


/tmp/ipykernel_214231/1808138414.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sampled_probabilities = [probabilities[i] for i in sampled_indices]


In [85]:
token_map.loc['and', :].sum()

0.0387494216430021

In [67]:
np.random.choice(5, size=2, replace=False, p=[0.5,0.2,0.1,0.1,0.1])

array([1, 0])